# Synthesis Rate Regressions

In [41]:
import cPickle as pkl
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

## 1. Get data

Regression data:

In [42]:
regression_data = pkl.load(open("../../parameters/regression_data_small.p", "rb"))

This is the data from notebook 09.

In [43]:
regression_data.columns = ['name', 'tic toc [s]', 'ORF length [nts]', 'elongation speed [nts/s]', 'transcripts', 
                           'synth. rate [molecules/s]', 'synth. rate per transcript [molecules/transcript/s]', 
                           'tAI', 'p_init', 'elongation rate [1/s]', 'time per initiation [s]', 
                           'time per protein [s]']

"Experimental" synthesis rates (https://www.ncbi.nlm.nih.gov/pubmed/12660367):

In [44]:
synth_arava = pd.DataFrame(pd.Series(pkl.load(open("../../parameters/prot_arava.p", "rb"))))
synth_arava.columns = ['synth. rate exp. [molecules/s]']

In [45]:
synth_arava.tail()

,synth. rate exp. [molecules/s]
YPR199C,0.079946
YPR200C,0.224314
YPR202W,0.002615
YPR203W,0.051853
YPR204W,0.001113


In [46]:
regression_data = pd.merge(regression_data, synth_arava, left_on='name', right_index=True, how='left')

In [47]:
regression_data['synth. rate exp. per transcript [molecules/s]'] = regression_data['synth. rate exp. [molecules/s]'] / regression_data['transcripts']

In [48]:
regression_data.tail()

,name,tic toc [s],ORF length [nts],elongation speed [nts/s],transcripts,synth. rate [molecules/s],synth. rate per transcript [molecules/transcript/s],tAI,p_init,elongation rate [1/s],time per initiation [s],time per protein [s],synth. rate exp. [molecules/s],synth. rate exp. per transcript [molecules/s]
4470,YJL081C,71.884435,1470,20.449490,5,0.150287,0.030057,0.375582,9.111658e-07,0.013911,1.097495e+06,33.269598,0.072179,0.014436
4471,YMR127C,60.685714,1017,16.758475,1,0.022414,0.022414,0.323240,5.499448e-07,0.016478,1.818364e+06,44.615385,0.062877,0.062877
4472,YPL013C,19.936467,366,18.358318,5,0.318984,0.063797,0.308066,2.289991e-06,0.050159,4.366829e+05,15.674775,NaN,NaN
4473,YLR440C,97.862222,2130,21.765294,2,0.072126,0.036063,0.368466,1.186173e-06,0.010218,8.430470e+05,27.729084,0.000000,0.000000
4474,YLR192C,26.893313,798,29.672804,22,2.742112,0.124641,0.510948,4.290874e-06,0.037184,2.330528e+05,8.023014,0.444057,0.020184


## 2. Linear regression setup

In [49]:
ntest = 100

Utility function to produce regression data from a data frame:

In [50]:
def regression_data_from_frame(frame, x_list, y, ntest=100):
    regression_x = np.array(frame[x_list])

    # Split the data into training/testing sets
    x_train = regression_x[:-ntest]
    x_test = regression_x[-ntest:]

    # Split the targets into training/testing sets
    y_train = np.array(frame[y][:-ntest])
    y_test = np.array(frame[y][-ntest:])
    return x_train, y_train, x_test, y_test

In [51]:
def build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True):
    # Create linear regression object
    regr = linear_model.LinearRegression(fit_intercept=fit_intercept)

    # Train the model using the training sets
    regr.fit(x_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(x_test)

    # The coefficients
    print 'Coefficients: ', regr.coef_
    print 'Intercept: ', regr.intercept_

    # The mean squared error
    print "Mean squared error: %.2f" % mean_squared_error(y_test, y_pred)

    # Explained variance score: 1 is perfect prediction
    print 'R² score: %.2f' % r2_score(y_test, y_pred)
    
    return regr

TODO: residuals!

## 3. Build some models

### 3.1 Univariate models against synthetic rates

#### 3.1.1 tAI

In [52]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data, 
                                                              ['tAI'], 
                                                              'synth. rate per transcript [molecules/transcript/s]', 
                                                              ntest=100)

In [53]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 0.38154187]
Intercept:  -0.102180635748
Mean squared error: 0.00
R² score: 0.52


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### 3.1.2 $p_I$

In [54]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data, 
                                                              ['p_init'], 
                                                              'synth. rate per transcript [molecules/transcript/s]', 
                                                              ntest=100)

In [55]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 27584.3509447]
Intercept:  0.00354864175085
Mean squared error: 0.00
R² score: 0.99


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [56]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=False)

Coefficients:  [ 29140.88338432]
Intercept:  0.0
Mean squared error: 0.00
R² score: 0.99


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

### 3.2 Univariate models against experimental rates

#### 3.2.1 tAI

In [57]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. per transcript [molecules/s]'].notnull()], 
                                                              ['tAI'], 
                                                              'synth. rate exp. per transcript [molecules/s]', 
                                                              ntest=100)

In [58]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [-0.10892593]
Intercept:  0.0803804666481
Mean squared error: 0.00
R² score: 0.03


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [59]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. [molecules/s]'].notnull()], 
                                                              ['tAI'], 
                                                              'synth. rate exp. [molecules/s]', 
                                                              ntest=100)

In [60]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 7.88581603]
Intercept:  -2.808006281
Mean squared error: 0.19
R² score: 0.43


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Interesting - the unnormalized works, the normalized does not; TODO: check Arava et al. (2003) paper!

#### 3.2.2 $p_I$

In [61]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. per transcript [molecules/s]'].notnull()], 
                                                              ['p_init'], 
                                                              'synth. rate exp. per transcript [molecules/s]', 
                                                              ntest=100)

In [62]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 4358.97672646]
Intercept:  0.0311422680025
Mean squared error: 0.00
R² score: -0.02


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [63]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. [molecules/s]'].notnull()], 
                                                              ['p_init'], 
                                                              'synth. rate exp. [molecules/s]', 
                                                              ntest=100)

In [64]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 412838.91734464]
Intercept:  -0.379220718624
Mean squared error: 0.18
R² score: 0.46


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [65]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=False)

Coefficients:  [ 246693.46750851]
Intercept:  0.0
Mean squared error: 0.21
R² score: 0.37


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

### 3.3 Multivariate models against synthetic rates

In [66]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data, 
                                                              ['p_init', 'tAI'], 
                                                              'synth. rate per transcript [molecules/transcript/s]', 
                                                              ntest=100)

In [67]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [  2.71259851e+04   1.12044668e-02]
Intercept:  -0.000103149288903
Mean squared error: 0.00
R² score: 0.99


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Standardize independent variables to find influence:

In [68]:
def normalize(df):
    return (df - df.mean()) / df.std()

In [69]:
regression_data['p_init_norm'] = normalize(regression_data['p_init'])
regression_data['tAI_norm'] = normalize(regression_data['tAI'])

In [70]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data, 
                                                              ['p_init_norm', 'tAI_norm'], 
                                                              'synth. rate per transcript [molecules/transcript/s]', 
                                                              ntest=100)

In [71]:
regr = build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 0.02875654  0.00065014]
Intercept:  0.046432147338
Mean squared error: 0.00
R² score: 0.99


In [72]:
regr.coef_[0] / regr.coef_[1]

44.231482812858033

The coefficient on $p_I$ is 44x bigger than the coefficient on tAI.

### 3.4 Multivariate models against experimental rates

In [73]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. [molecules/s]'].notnull()], 
                                                              ['p_init', 'tAI'], 
                                                              'synth. rate exp. per transcript [molecules/s]', 
                                                              ntest=100)

In [74]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [  2.01318090e+04  -3.85999557e-01]
Intercept:  0.156943809494
Mean squared error: 0.00
R² score: 0.07


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [75]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. [molecules/s]'].notnull()], 
                                                              ['p_init', 'tAI'], 
                                                              'synth. rate exp. [molecules/s]', 
                                                              ntest=100)

In [76]:
build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [  2.07046596e+05   5.03623847e+00]
Intercept:  -2.02058675313
Mean squared error: 0.16
R² score: 0.50


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [77]:
x_train, y_train, x_test, y_test = regression_data_from_frame(regression_data[regression_data['synth. rate exp. [molecules/s]'].notnull()], 
                                                              ['p_init_norm', 'tAI_norm'], 
                                                              'synth. rate exp. [molecules/s]', 
                                                              ntest=100)

In [78]:
regr = build_linear_model(x_train, y_train, x_test, y_test, fit_intercept=True)

Coefficients:  [ 0.21949223  0.29222688]
Intercept:  0.262591920129
Mean squared error: 0.16
R² score: 0.50


In [79]:
regr.coef_[0] / regr.coef_[1]

0.75110211304803087

Approximately the same influence by both variables!

In [80]:
pkl.dump(regression_data, open("../../parameters/regression_data.p", "wb"))

## TODO: residuals and images!